In [1]:
import redis
import os
from dotenv import load_dotenv

from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

### We will be using the Titan Embeddings Model to generate our Embeddings.

In [2]:

from langchain_aws.embeddings import BedrockEmbeddings
from langchain_aws.llms.bedrock import Bedrock
load_dotenv()

False

###  Define  the Anthropic Model params 

In [3]:
# - create the Anthropic Model
model_kwargs = {"max_tokens_to_sample": 8000,
            "temperature": 0, 
            "top_k": 250, 
            "top_p": 1,
            "stop_sequences": ["\\n\\nHuman:"]
               }    

### Initialize large language model and use model properties for Claude-v2

In [4]:
llm = Bedrock(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to use for AWS credentials (if not the default)
        region_name=os.environ.get("BWB_REGION_NAME"), #sets the region name (if not the default)
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"), #sets the endpoint URL (if necessary)
        model_id="anthropic.claude-v2", #use the Anthropic Claude model
        model_kwargs=model_kwargs
) #configure the properties for Claude


/home/ec2-user/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockLLM`.
  warn_deprecated(


### Define titan embeddings model

In [5]:
embeddings = BedrockEmbeddings(
        credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to use for AWS credentials (if not the default)
        region_name=os.environ.get("BWB_REGION_NAME"), #sets the region name (if not the default)
        endpoint_url=os.environ.get("BWB_ENDPOINT_URL"), #sets the endpoint URL (if necessary)
    ) #create a Titan Embeddings client
  

##### Here is the document we load for using in context. 


In [6]:
pdf_path = "/home/ec2-user/environment/memorydb-guide.pdf"
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

[Go to Section Title](#section_name)


### Pre process the data to split into chunks that can be loaded into Vector database. 

In [7]:
%%time
loader = PyPDFLoader(file_path=pdf_path) #load the pdf file
pages=loader.load_and_split()
#pages[10] Uncomment if you want to see the data 

text_splitter = RecursiveCharacterTextSplitter( #create a text splitter
        separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sentence, or (4) word, in that order
        chunk_size=1000, #divide into 1000-character chunks using the separators above
        chunk_overlap=100 #number of characters that can overlap with previous chunk
    )
chunks = loader.load_and_split(text_splitter)

CPU times: user 45.4 s, sys: 77.1 ms, total: 45.5 s
Wall time: 45.5 s


## Using MemoryDB as Vector store. 
We test out both Semantic Search and using MemoryDB as retriever for RAG. 
We are using MemoryDB for our vector store. This code tests connection to MemDB and clears the existing data. 

**Comment client_devo.flushall() if you dont want to clear the data and index creation**


In [8]:
from redis.cluster import RedisCluster as MemoryDBCluster

In [9]:
rc = MemoryDBCluster(host='Your_cluster_Endpoint', 
           port=6379,ssl=True, decode_responses=True, ssl_cert_reqs="none")

rc.ping()
rc.flushall()

True

In [10]:
INDEX_NAME='idx:vss-mm'
MEMORYDB_url="rediss://Your_cluster_Endpoint:6379/ssl=True&ssl_cert_reqs=none",


In [11]:
vector_schema = {
    "algorithm": "HNSW"
}


### Create the index and Load the documents with their embeddings into Redis. 

In [12]:
%%time
vds = InMemoryVectorStore.from_documents(
            chunks,
            embeddings,
            redis_url="rediss://Your_cluster_Endpoint:6379/ssl=True&ssl_cert_reqs=none",
            vector_schema=vector_schema,
            index_name=INDEX_NAME,
        )

CPU times: user 2.05 s, sys: 150 ms, total: 2.2 s
Wall time: 1min 46s


### Lets inspect the index we created 

In [13]:
%%time
info = rc.ft(INDEX_NAME).info()
num_docs = info['num_docs']
space_usage = info['space_usage']
num_indexed_vectors = info['num_indexed_vectors']
vector_space_usage = (info['vector_space_usage'])

print(f"{num_docs} documents ({space_usage} space used vectors indexed {num_indexed_vectors} vector space usage in {vector_space_usage}")

797 documents (1553039 space used vectors indexed 797 vector space usage in 886996
CPU times: user 2.23 ms, sys: 0 ns, total: 2.23 ms
Wall time: 2 ms


### Testing similarity search 

[Here are some search functions](https://python.langchain.com/docs/integrations/vectorstores/redis#querying)!

## Run this if the index is already created and data is loaded 

In [14]:
%%time
vds = InMemoryVectorStore(
        redis_url="rediss://Your_cluster_Endpoint:6379/ssl=True&ssl_cert_reqs=none",
        index_name=INDEX_NAME,
        embedding=embeddings,
       # index_schema=index_schema,  # Replace with your index schema if needed
)


CPU times: user 146 ms, sys: 9.73 ms, total: 156 ms
Wall time: 171 ms


In [15]:
query = "how to do backups with memoryDB "


In [16]:
%%time
results = vds.similarity_search(query)
(results)

CPU times: user 4.93 ms, sys: 0 ns, total: 4.93 ms
Wall time: 86.7 ms


[Document(page_content="console.aws.amazon.com/memorydb/.\n2. from the left navigation pane, choose Clusters .\nThe MemoryDB clusters screen appears.\n3. choose the radio button to the left of the name of the MemoryDB cluster you want to back up.\n4. Choose Actions and then Take snapshot.\n5. In the Snapshot  window, type in a name for your snapshot in the Snapshot Name  box. We \nrecommend that the name indicate which cluster was backed up and the date and time the snapshot \nwas made.\nCluster naming constraints are as follows:\n•Must contain 1–40 alphanumeric characters or hyphens.\n•Must begin with a letter.\n•Can't contain two consecutive hyphens.\n•Can't end with a hyphen.\n6. Under Encryption, choose whether to use a default encryption key or a customer managed key. For \nmore information, see In-transit encryption (TLS) in MemoryDB (p. 195).\n7. Under Tags, optionally add tags to search and ﬁlter your snapshots or track your AWS costs.\n130", metadata={'id': 'doc:idx:vss-mm:1ee

In [17]:
for item in results:
    print(item.page_content, end="\n\n")

console.aws.amazon.com/memorydb/.
2. from the left navigation pane, choose Clusters .
The MemoryDB clusters screen appears.
3. choose the radio button to the left of the name of the MemoryDB cluster you want to back up.
4. Choose Actions and then Take snapshot.
5. In the Snapshot  window, type in a name for your snapshot in the Snapshot Name  box. We 
recommend that the name indicate which cluster was backed up and the date and time the snapshot 
was made.
Cluster naming constraints are as follows:
•Must contain 1–40 alphanumeric characters or hyphens.
•Must begin with a letter.
•Can't contain two consecutive hyphens.
•Can't end with a hyphen.
6. Under Encryption, choose whether to use a default encryption key or a customer managed key. For 
more information, see In-transit encryption (TLS) in MemoryDB (p. 195).
7. Under Tags, optionally add tags to search and ﬁlter your snapshots or track your AWS costs.
130

Amazon MemoryDB for Redis Developer Guide
Exporting a snapshot
4. Choose Cop

## RAG 

### RAG
The below code helps implement MemoryDB vector database as a retriever.By default, it will use [**semantic similarity**](https://python.langchain.com/docs/integrations/vectorstores/redis#redis-as-retriever)!

[Retreival Augmented Generation](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html)!

We will use MemoryDB developer Guide 

In [18]:
%%time
retriever=vds.as_retriever()

CPU times: user 102 µs, sys: 0 ns, total: 102 µs
Wall time: 105 µs


In [20]:
%%time
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to provide a concise answer in English to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa_prompt = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vds.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    #verbose="true"
)
query = "How do i create a MemoryDB cluster"
result = qa_prompt({"query": query})
print(result["result"])

/home/ec2-user/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


 Based on the provided context, to create a MemoryDB cluster you can use the AWS Management Console, AWS CLI, or MemoryDB API. 

Some key steps are:

- Sign in to the AWS Management Console and open the MemoryDB console at https://console.aws.amazon.com/memorydb/
- Choose Clusters in the left navigation pane and then choose Create Cluster. 
- Specify a name, description, subnet group, and other configuration options.
- Review the settings and launch the cluster.

The context provides more detailed instructions on using the console, CLI, and API to create a cluster. But in summary, you need to sign in to the MemoryDB console or use the CLI/API and specify cluster details like name, network, parameter groups etc to launch a new cluster.
CPU times: user 59.7 ms, sys: 108 µs, total: 59.8 ms
Wall time: 5.76 s


In [22]:
%%time
retriever=vds.as_retriever()

CPU times: user 98 µs, sys: 2 µs, total: 100 µs
Wall time: 103 µs
